In [2]:
import pandas as pd

og_df = pd.read_pickle('df.pkl') 
count_df1 = og_df.groupby(['transcript_id', 'transcript_position']).size().reset_index(name='read_count')
count_df2 = og_df.groupby(['transcript_id']).size().reset_index(name='expression_count') #most genes are likely to produce unique transcripts
df = og_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
df = df.merge(count_df2, on=['transcript_id'])

agg_functions = {
    'gene_id': 'first',
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
    'label': 'sum'
}

summary_df = og_df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()

summary_df = summary_df.merge(count_df1, on=['transcript_id', 'transcript_position'])
summary_df = summary_df.merge(count_df2, on=['transcript_id'])

summary_df['label_percentage'] = summary_df['label'] / summary_df['read_count'] * 100
summary_df = summary_df.rename(columns={'label': 'label_sum'})
summary_df['label'] = summary_df['label_percentage'].apply(lambda x: 0 if x == 0 else 1)

In [3]:
agg_functions = {
    'gene_id': 'first',
    'combined nucleotides': 'first',
    'dwelling_time1': 'mean',
    'sd1': 'mean',
    'mean1': 'mean',
    'dwelling_time2': 'mean',
    'sd2': 'mean',
    'mean2': 'mean',
    'dwelling_time3': 'mean',
    'sd3': 'mean',
    'mean3': 'mean',
    'label': 'first'
}

summary_df1 = og_df.groupby(['transcript_id', 'transcript_position']).agg(agg_functions).reset_index()
summary_df1['label'].equals(summary_df['label'])

True

In [4]:
summary_df['label_percentage'].unique()

array([  0., 100.])

In [5]:
summary_df = summary_df.drop(columns=['label_percentage', 'label_sum'])
summary_df

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,expression_count,label
0,ENST00000000233,244,ENSG00000004059,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,4.386989,80.570270,185,3449,0
1,ENST00000000233,261,ENSG00000004059,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,3.016599,94.290698,172,3449,0
2,ENST00000000233,316,ENSG00000004059,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,2.087146,89.364324,185,3449,0
3,ENST00000000233,332,ENSG00000004059,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,2.236520,89.154000,200,3449,0
4,ENST00000000233,368,ENSG00000004059,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,4.260253,85.178788,198,3449,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,ENSG00000167747,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,4.005616,82.004110,73,1323,1
121834,ENST00000641834,1429,ENSG00000167747,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,3.644638,80.497101,69,1323,0
121835,ENST00000641834,1531,ENSG00000167747,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,2.181563,84.190625,64,1323,1
121836,ENST00000641834,1537,ENSG00000167747,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,2.540877,82.289474,57,1323,0


In [6]:
print(df['label'].value_counts())
print(summary_df['label'].value_counts())

label
0    10527532
1      499574
Name: count, dtype: int64
label
0    116363
1      5475
Name: count, dtype: int64


### 5-mer feature and binary features for first and last letters

In [7]:
# function to convert features
def sequence_to_features(seq):
    """
    Convert a 7-letter sequence into binary features for the first and last letters.
    
    Parameters:
    - seq: A string of 7 letters (A, T, C, G)
    
    Returns:
    - A dictionary with binary features for the first and last letters.
    """
    letters = ['A', 'T', 'C', 'G']
    features = {}
    
    first_char = seq[0]
    last_char = seq[-1]
    
    for letter in letters:
        features[f"1{letter}"] = first_char == letter
        features[f"7{letter}"] = last_char == letter
            
    return features

In [8]:
new_df = summary_df.copy()

feature_dicts = new_df['combined nucleotides'].apply(sequence_to_features)
features_df = feature_dicts.apply(pd.Series)
new_df = pd.concat([new_df, features_df], axis=1)

In [9]:
new_df

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,...,expression_count,label,1A,7A,1T,7T,1C,7C,1G,7G
0,ENST00000000233,244,ENSG00000004059,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,...,3449,0,True,True,False,False,False,False,False,False
1,ENST00000000233,261,ENSG00000004059,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,...,3449,0,False,False,False,False,True,False,False,True
2,ENST00000000233,316,ENSG00000004059,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,...,3449,0,False,False,False,False,False,False,True,True
3,ENST00000000233,332,ENSG00000004059,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,...,3449,0,True,False,False,True,False,False,False,False
4,ENST00000000233,368,ENSG00000004059,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,...,3449,0,True,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,ENSG00000167747,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,...,1323,1,False,False,False,True,False,False,True,False
121834,ENST00000641834,1429,ENSG00000167747,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,...,1323,0,False,False,False,False,True,True,False,False
121835,ENST00000641834,1531,ENSG00000167747,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,...,1323,1,False,False,True,False,False,True,False,False
121836,ENST00000641834,1537,ENSG00000167747,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,...,1323,0,False,True,False,False,True,False,False,False


In [10]:
new_df['5-mer'] = new_df['combined nucleotides'].apply(lambda x: x[1:-1])
new_df.groupby(['5-mer'])['label'].sum().sort_values(ascending=False)

5-mer
GGACT    1440
GAACT     783
GGACA     691
GGACC     552
AGACT     525
TGACT     430
AAACT     255
GAACA     187
AGACA     147
GAACC     124
TGACC      90
TAACT      71
TGACA      68
AGACC      60
AAACA      38
AAACC      11
TAACC       2
TAACA       1
Name: label, dtype: int64

In [11]:
new_df

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,...,label,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,ENST00000000233,244,ENSG00000004059,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,...,0,True,True,False,False,False,False,False,False,AGACC
1,ENST00000000233,261,ENSG00000004059,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,...,0,False,False,False,False,True,False,False,True,AAACT
2,ENST00000000233,316,ENSG00000004059,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,...,0,False,False,False,False,False,False,True,True,AAACA
3,ENST00000000233,332,ENSG00000004059,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,...,0,True,False,False,True,False,False,False,False,GAACA
4,ENST00000000233,368,ENSG00000004059,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,...,0,True,True,False,False,False,False,False,False,GGACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,ENST00000641834,1348,ENSG00000167747,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,...,1,False,False,False,True,False,False,True,False,GGACA
121834,ENST00000641834,1429,ENSG00000167747,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,...,0,False,False,False,False,True,True,False,False,TGACA
121835,ENST00000641834,1531,ENSG00000167747,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,...,1,False,False,True,False,False,True,False,False,GGACA
121836,ENST00000641834,1537,ENSG00000167747,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,...,0,False,True,False,False,True,False,False,False,TGACC


### Label Encoder

In [12]:
from sklearn.preprocessing import LabelEncoder

combined = new_df.copy()

label_encoder = LabelEncoder()
combined['transcript_id'] = label_encoder.fit_transform(combined['transcript_id'])
combined['gene_id'] = label_encoder.fit_transform(combined['gene_id'])
combined

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,...,label,1A,7A,1T,7T,1C,7C,1G,7G,5-mer
0,0,244,10,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,...,0,True,True,False,False,False,False,False,False,AGACC
1,0,261,10,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,...,0,False,False,False,False,True,False,False,True,AAACT
2,0,316,10,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,...,0,False,False,False,False,False,False,True,True,AAACA
3,0,332,10,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,...,0,True,False,False,True,False,False,False,False,GAACA
4,0,368,10,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,...,0,True,True,False,False,False,False,False,False,GGACA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,...,1,False,False,False,True,False,False,True,False,GGACA
121834,5332,1429,2764,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,...,0,False,False,False,False,True,True,False,False,TGACA
121835,5332,1531,2764,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,...,1,False,False,True,False,False,True,False,False,GGACA
121836,5332,1537,2764,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,...,0,False,True,False,False,True,False,False,False,TGACC


In [13]:
combined = pd.get_dummies(combined, columns=['5-mer'], drop_first=False)
combined

,transcript_id,transcript_position,gene_id,combined nucleotides,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,10,AAGACCA,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,...,False,False,False,False,False,False,False,False,False,False
1,0,261,10,CAAACTG,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,...,False,False,False,False,False,False,False,False,False,False
2,0,316,10,GAAACAG,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,...,False,False,False,False,False,False,False,False,False,False
3,0,332,10,AGAACAT,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,...,False,False,False,False,False,False,False,False,False,False
4,0,368,10,AGGACAA,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,GGGACAT,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,...,False,True,False,False,False,False,False,False,False,False
121834,5332,1429,2764,CTGACAC,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,...,False,False,False,False,False,False,False,True,False,False
121835,5332,1531,2764,TGGACAC,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,...,False,True,False,False,False,False,False,False,False,False
121836,5332,1537,2764,CTGACCA,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,...,False,False,False,False,False,False,False,False,True,False


In [14]:
combined = combined.drop(columns=['combined nucleotides'])
combined

,transcript_id,transcript_position,gene_id,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,10,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,False,False,False,False,False,False,False,False,False,False
1,0,261,10,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,False,False,False,False,False,False,False,False,False,False
2,0,316,10,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,False,False,False,False,False,False,False,False,False,False
3,0,332,10,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,False,False,False,False,False,False,False,False,False,False
4,0,368,10,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,False,True,False,False,False,False,False,False,False,False
121834,5332,1429,2764,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,False,False,False,False,False,False,False,True,False,False
121835,5332,1531,2764,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,False,True,False,False,False,False,False,False,False,False
121836,5332,1537,2764,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,False,False,False,False,False,False,False,False,True,False


In [15]:
combined.value_counts('gene_id')

gene_id
1528    274
977     253
2632    250
2005    230
1929    229
       ... 
857       1
863       1
887       1
2800      1
1412      1
Name: count, Length: 3852, dtype: int64

In [16]:
combined.describe()

,transcript_id,transcript_position,gene_id,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,sd3,mean3,read_count,expression_count,label
count,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000
mean,2474.902666,1615.868407,1846.301105,0.008108,4.342127,110.789403,0.008127,5.157547,110.946258,0.007057,2.974051,86.203293,90.506295,3136.770769,0.044937
std,1503.787527,1395.137528,1098.013215,0.001787,1.936267,11.289521,0.001693,2.184069,12.460242,0.001296,0.894199,4.937330,137.333376,4732.533068,0.207166
min,0.000000,3.000000,0.000000,0.003616,1.361306,80.370000,0.003803,1.441381,87.071498,0.003537,1.350225,72.231429,20.000000,20.000000,0.000000
25%,1162.000000,673.000000,895.000000,0.006799,2.853595,104.596000,0.006904,3.221361,98.816048,0.006151,2.282982,82.050397,32.000000,873.000000,0.000000
50%,2399.000000,1256.000000,1847.000000,0.007760,3.835318,110.122035,0.007834,4.669024,114.536673,0.006822,2.814199,86.500000,47.000000,1752.000000,0.000000
75%,3663.000000,2139.000000,2776.000000,0.009101,5.420603,119.299837,0.009066,6.963261,121.552632,0.007714,3.494345,89.997368,84.000000,3500.000000,0.000000
max,5332.000000,18730.000000,3851.000000,0.019119,11.644615,136.285714,0.021581,12.128800,137.120000,0.016344,11.898750,105.773134,991.000000,68776.000000,1.000000


### Resampling Methods

In [17]:
X = combined.drop('label', axis=1)
y = combined['label']

In [18]:
X

,transcript_id,transcript_position,gene_id,dwelling_time1,sd1,mean1,dwelling_time2,sd2,mean2,dwelling_time3,...,5-mer_GAACT,5-mer_GGACA,5-mer_GGACC,5-mer_GGACT,5-mer_TAACA,5-mer_TAACC,5-mer_TAACT,5-mer_TGACA,5-mer_TGACC,5-mer_TGACT
0,0,244,10,0.008264,4.223784,123.702703,0.009373,7.382162,125.913514,0.007345,...,False,False,False,False,False,False,False,False,False,False
1,0,261,10,0.006609,3.216424,109.681395,0.006813,3.226535,107.889535,0.007710,...,False,False,False,False,False,False,False,False,False,False
2,0,316,10,0.007570,2.940541,105.475676,0.007416,3.642703,98.947027,0.007555,...,False,False,False,False,False,False,False,False,False,False
3,0,332,10,0.010620,6.476350,129.355000,0.008632,2.899200,97.836500,0.006101,...,False,False,False,False,False,False,False,False,False,False
4,0,368,10,0.010701,6.415051,117.924242,0.011479,5.870303,121.954545,0.010019,...,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121833,5332,1348,2764,0.009594,3.294164,118.232877,0.007300,4.929726,116.342466,0.006555,...,False,True,False,False,False,False,False,False,False,False
121834,5332,1429,2764,0.008393,4.511014,110.969565,0.010305,9.105797,114.927536,0.005568,...,False,False,False,False,False,False,False,True,False,False
121835,5332,1531,2764,0.008161,3.918438,113.968750,0.006877,4.759688,113.562500,0.006410,...,False,True,False,False,False,False,False,False,False,False
121836,5332,1537,2764,0.008044,3.191228,109.354386,0.007419,6.552982,123.263158,0.006472,...,False,False,False,False,False,False,False,False,True,False


In [19]:
from imblearn.under_sampling import TomekLinks, RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTETomek

# ADASYN followed by Tomek Links Pipeline
adasyn_tomek = Pipeline([
    ('adasyn', ADASYN(random_state=42)),
    ('tomek', TomekLinks())
])

# Dictionary of resampling methods
resampling_methods = {
    'RandomUnderSampler': RandomUnderSampler(random_state=42),
    'TomekLinks': TomekLinks(),
    'SMOTE': SMOTE(random_state=42),
    'ADASYN': ADASYN(random_state=42),
    'SMOTETomek': SMOTETomek(random_state=42),
    'ADASYN_Tomek': adasyn_tomek
}

### 5-mer & +1 -1

In [20]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, precision_recall_curve

# Initialize GroupShuffleSplit
group_shuffle_split = GroupShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Model initialization
xgb_model_1 = xgb.XGBClassifier(n_estimators=200, random_state=42, learning_rate=0.1, max_depth=5)

# Drop 'gene_id' from X
X_min_features_1 = X.drop(columns=['gene_id', 'transcript_id', 'transcript_position'])


In [22]:
from sklearn.metrics import auc, precision_recall_curve

roc_aucs = []  # List to store ROC AUC scores for each fold
pr_aucs = []   # List to store PR AUC scores for each fold
avg_scores = [] # List to store the average of ROC AUC and PR AUC scores for each fold

# Loop through each fold
for train_index, valid_index in group_shuffle_split.split(X, y, groups=X['gene_id']):
    X_train_fold, X_valid_fold = X_min_features_1.iloc[train_index], X_min_features_1.iloc[valid_index]
    y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

    # Separate boolean and non-boolean columns
    X_train_fold_non_bool = X_train_fold.select_dtypes(exclude=['bool'])
    X_train_fold_bool = X_train_fold.select_dtypes(include=['bool'])
    
    X_valid_fold_non_bool = X_valid_fold.select_dtypes(exclude=['bool'])
    X_valid_fold_bool = X_valid_fold.select_dtypes(include=['bool'])

    # Initialize the scaler
    scaler = MinMaxScaler()

    # Fit the scaler on the training data and transform both training and validation data
    X_train_fold_scaled_non_bool = scaler.fit_transform(X_train_fold_non_bool)
    X_valid_fold_scaled_non_bool = scaler.transform(X_valid_fold_non_bool)

    # Concatenate the scaled non-boolean columns with the boolean columns
    X_train_fold_final = np.hstack([X_train_fold_scaled_non_bool, X_train_fold_bool.values])
    X_valid_fold_final = np.hstack([X_valid_fold_scaled_non_bool, X_valid_fold_bool.values])

    # Apply the current resampling technique
    X_resampled, y_resampled = TomekLinks().fit_resample(X_train_fold_final, y_train_fold)

    # Train the model on the resampled data
    xgb_model_1.fit(X_resampled, y_resampled)

    # Predict on the validation set
    y_pred = xgb_model_1.predict_proba(X_valid_fold_final)[:, 1]

    # Compute the ROC AUC score for this fold and append to the list
    roc_auc = roc_auc_score(y_valid_fold, y_pred)
    roc_aucs.append(roc_auc)

    # Compute Precision-Recall curve and PR AUC score for this fold
    precision, recall, _ = precision_recall_curve(y_valid_fold, y_pred)
    pr_auc = auc(recall, precision)
    pr_aucs.append(pr_auc)
    
    # Compute the average of ROC AUC and PR AUC scores for this fold
    avg_score = (roc_auc + pr_auc) / 2
    avg_scores.append(avg_score)

# Display results for the ROC AUC scores
print("ROC AUC scores for each fold:", roc_aucs)
print("Mean ROC AUC score:", np.mean(roc_aucs))
print("Standard deviation of ROC AUC scores:", np.std(roc_aucs))

print("-" * 50)

# Display results for the PR AUC scores
print("PR AUC scores for each fold:", pr_aucs)
print("Mean PR AUC score:", np.mean(pr_aucs))
print("Standard deviation of PR AUC scores:", np.std(pr_aucs))

print("-" * 50)

# Display results for the average scores
print("Average of ROC AUC and PR AUC scores for each fold:", avg_scores)
print("Mean average score:", np.mean(avg_scores))
print("Standard deviation of average scores:", np.std(avg_scores))

print("-" * 50)

ROC AUC scores for each fold: [0.8981943546115879, 0.9046156698106628, 0.898218066165208, 0.8983467657756298, 0.8953642374203081]
Mean ROC AUC score: 0.8989478187566793
Standard deviation of ROC AUC scores: 0.003047231612898998
--------------------------------------------------
PR AUC scores for each fold: [0.4775251240597655, 0.49198502884775736, 0.4551578475988918, 0.4288276327957216, 0.4484863170211557]
Mean PR AUC score: 0.4603963900646583
Standard deviation of PR AUC scores: 0.022168059143806432
--------------------------------------------------
Average of ROC AUC and PR AUC scores for each fold: [0.6878597393356767, 0.69830034932921, 0.6766879568820499, 0.6635871992856757, 0.6719252772207319]
Mean average score: 0.6796721044106688
Standard deviation of average scores: 0.012179421293917905
--------------------------------------------------


In [23]:
print(xgb_model_1)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)


In [24]:
predictors_scaled = scaler.fit_transform(combined[X_min_features_1.columns])
pd.DataFrame(predictors_scaled).describe()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
count,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,...,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000,121838.000000
mean,0.289714,0.289870,0.544022,0.243220,0.347714,0.477032,0.274858,0.153939,0.416552,0.072612,...,0.057100,0.063420,0.060351,0.052332,0.038535,0.024344,0.036384,0.063346,0.059948,0.055722
std,0.115253,0.188292,0.201902,0.095204,0.204359,0.248963,0.101173,0.084770,0.147200,0.141435,...,0.232035,0.243718,0.238136,0.222696,0.192484,0.154115,0.187245,0.243586,0.237392,0.229384
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.205320,0.145118,0.433259,0.174416,0.166549,0.234663,0.204172,0.088425,0.292739,0.012358,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.267269,0.240585,0.532087,0.226743,0.302004,0.548771,0.256523,0.138785,0.425398,0.027806,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.353792,0.394746,0.696224,0.296027,0.516671,0.688954,0.326192,0.203263,0.529667,0.065911,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [25]:
X_min_features_1.columns

Index(['dwelling_time1', 'sd1', 'mean1', 'dwelling_time2', 'sd2', 'mean2',
       'dwelling_time3', 'sd3', 'mean3', 'read_count', 'expression_count',
       '1A', '7A', '1T', '7T', '1C', '7C', '1G', '7G', '5-mer_AAACA',
       '5-mer_AAACC', '5-mer_AAACT', '5-mer_AGACA', '5-mer_AGACC',
       '5-mer_AGACT', '5-mer_GAACA', '5-mer_GAACC', '5-mer_GAACT',
       '5-mer_GGACA', '5-mer_GGACC', '5-mer_GGACT', '5-mer_TAACA',
       '5-mer_TAACC', '5-mer_TAACT', '5-mer_TGACA', '5-mer_TGACC',
       '5-mer_TGACT'],
      dtype='object')

In [26]:
y_pred = xgb_model_1.predict_proba(predictors_scaled)[:, 1]

In [27]:
y_pred

array([0.00518603, 0.00991507, 0.01354187, ..., 0.5684651 , 0.00922662,
       0.00411102], dtype=float32)

In [28]:
roc_auc_score(y, y_pred)

0.9328439326831792

In [29]:
from sklearn.metrics import average_precision_score
average_precision_score(y, y_pred)

0.5401812373746471

In [30]:
import pickle

# Export the trained model to a .pkl file
with open('xgb_model_1.pkl', 'wb') as model_file:
    pickle.dump(xgb_model_1, model_file)

### 5-mer & +1 -1 & transcript_position

In [31]:
# Initialize GroupShuffleSplit
group_shuffle_split = GroupShuffleSplit(n_splits=5, test_size=0.2, random_state=42)

# Model initialization
xgb_model_2 = xgb.XGBClassifier(n_estimators=200, random_state=42, learning_rate=0.1, max_depth=5)

# Drop 'gene_id' from X
X_min_features_2 = X.drop(columns=['gene_id', 'transcript_id'])

roc_aucs = []  # List to store ROC AUC scores for each fold
pr_aucs = []   # List to store PR AUC scores for each fold
avg_scores = [] # List to store the average of ROC AUC and PR AUC scores for each fold

# Loop through each fold
for train_index, valid_index in group_shuffle_split.split(X, y, groups=X['gene_id']):
    X_train_fold, X_valid_fold = X_min_features_2.iloc[train_index], X_min_features_2.iloc[valid_index]
    y_train_fold, y_valid_fold = y.iloc[train_index], y.iloc[valid_index]

    # Separate boolean and non-boolean columns
    X_train_fold_non_bool = X_train_fold.select_dtypes(exclude=['bool'])
    X_train_fold_bool = X_train_fold.select_dtypes(include=['bool'])
    
    X_valid_fold_non_bool = X_valid_fold.select_dtypes(exclude=['bool'])
    X_valid_fold_bool = X_valid_fold.select_dtypes(include=['bool'])

    # Initialize the scaler
    scaler = MinMaxScaler()

    # Fit the scaler on the non-boolean training data and transform both training and validation data
    X_train_fold_scaled_non_bool = scaler.fit_transform(X_train_fold_non_bool)
    X_valid_fold_scaled_non_bool = scaler.transform(X_valid_fold_non_bool)

    # Concatenate the scaled non-boolean columns with the boolean columns
    X_train_fold_final = np.hstack([X_train_fold_scaled_non_bool, X_train_fold_bool])
    X_valid_fold_final = np.hstack([X_valid_fold_scaled_non_bool, X_valid_fold_bool])

    # Apply the current resampling technique
    X_resampled, y_resampled = TomekLinks().fit_resample(X_train_fold_final, y_train_fold)

    # Train the model on the resampled data
    xgb_model_2.fit(X_resampled, y_resampled)

    # Predict on the validation set
    y_pred = xgb_model_2.predict_proba(X_valid_fold_final)[:, 1]

    # Compute the ROC AUC score for this fold and append to the list
    roc_auc = roc_auc_score(y_valid_fold, y_pred)
    roc_aucs.append(roc_auc)

    # Compute Precision-Recall curve and PR AUC score for this fold
    precision, recall, _ = precision_recall_curve(y_valid_fold, y_pred)
    pr_auc = auc(recall, precision)
    pr_aucs.append(pr_auc)
    
    # Compute the average of ROC AUC and PR AUC scores for this fold
    avg_score = (roc_auc + pr_auc) / 2
    avg_scores.append(avg_score)

# Display results for the ROC AUC scores
print("ROC AUC scores for each fold:", roc_aucs)
print("Mean ROC AUC score:", np.mean(roc_aucs))
print("Standard deviation of ROC AUC scores:", np.std(roc_aucs))

print("-" * 50)

# Display results for the PR AUC scores
print("PR AUC scores for each fold:", pr_aucs)
print("Mean PR AUC score:", np.mean(pr_aucs))
print("Standard deviation of PR AUC scores:", np.std(pr_aucs))

print("-" * 50)

# Display results for the average scores
print("Average of ROC AUC and PR AUC scores for each fold:", avg_scores)
print("Mean average score:", np.mean(avg_scores))
print("Standard deviation of average scores:", np.std(avg_scores))

print("-" * 50)

ROC AUC scores for each fold: [0.895790068122476, 0.9041627232774436, 0.8999807600376131, 0.8988597403742168, 0.8970830875376545]
Mean ROC AUC score: 0.8991752758698809
Standard deviation of ROC AUC scores: 0.0028795792243760145
--------------------------------------------------
PR AUC scores for each fold: [0.47541871985181466, 0.49522744721013334, 0.4611314096133351, 0.426972604753511, 0.4550047928409726]
Mean PR AUC score: 0.4627509948539533
Standard deviation of PR AUC scores: 0.022618183456774927
--------------------------------------------------
Average of ROC AUC and PR AUC scores for each fold: [0.6856043939871453, 0.6996950852437884, 0.6805560848254741, 0.6629161725638639, 0.6760439401893136]
Mean average score: 0.680963135361917
Standard deviation of average scores: 0.012020684385865012
--------------------------------------------------


In [32]:
print(xgb_model_2)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)


In [33]:
import pickle

# Export the trained model to a .pkl file
with open('xgb_model_2.pkl', 'wb') as model_file:
    pickle.dump(xgb_model_2, model_file)

In [34]:
predictors_scaled_2 = scaler.fit_transform(combined[X_min_features_2.columns])
y_pred_2 = xgb_model_2.predict_proba(predictors_scaled_2)[:, 1]
roc_auc_score(y, y_pred_2)


0.9363198426024497

In [35]:
average_precision_score(y, y_pred_2)

0.5527315851687131